In [20]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

os.environ.get("open_ai_key")

In [21]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import (
    Tool,
    agent_iterator,
    AgentExecutor,
    initialize_agent,
    create_react_agent,
)
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.52-turbo-preview", temperature=0)  # use

In [23]:
template = """ 
Answer the following questions as best as you can
Questions: {q}

"""
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull("hwchase17/react")
print(type(prompt))

print(prompt.input_variables)
print(prompt.template)

<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [30]:
pip install -q wikipedia


Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install -U duckduckgo-search

  Using cached duckduckgo_search-6.2.6-py3-none-any.whl.metadata (24 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached primp-0.5.5-cp38-abi3-win_amd64.whl.metadata (11 kB)
Using cached duckduckgo_search-6.2.6-py3-none-any.whl (27 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached primp-0.5.5-cp38-abi3-win_amd64.whl (3.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Python REPL tool
from langchain.agents import (
    Tool,
    agent_iterator,
    AgentExecutor,
    initialize_agent,
    create_react_agent,
)
from langchain_experimental.tools.python.tool import PythonAstREPLTool

python_repl = PythonAstREPLTool()
python_repl_tool = Tool(
    name="python Repl",
    func=python_repl.run,
    description="Useful when you need to use python to answer question. Please input python code",
)

# Wikipedia
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name="Wikipedia", func=wikipedia.run, description="useful for look up in wikipedia"
)
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGo Search",
    func=search.run,
    description="useful for performing web search",
)

In [ ]:
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10,
)

In [ ]:
Question = "generate the first 20 numbers in fibonacci series "
output = agent_executor.invoke({"input": prompt_template.format(q=Question)})